In [8]:
# ===========================
# SISTEMA DE PREDICCIÓN DE CORNERS - OPTIMIZADO PARA APUESTAS (VERSIÓN COMPLETA)
# ===========================

import numpy as np
import pandas as pd
import joblib
from scipy.stats import poisson

# ===========================
# 1. CARGAR MODELO Y SCALER
# ===========================

print("\n" + "=" * 80)
print("🎯 SISTEMA DE PREDICCIÓN DE CORNERS PARA APUESTAS (VERSIÓN AVANZADA)")
print("=" * 80)

xgb_model = joblib.load('models/xgboost_corners_optimized_v2_6_leagues.pkl')  # ✅ CAMBIO: versión v2
scaler = joblib.load('models/scaler_corners_xgb_v2_6_leagues.pkl')

print("✅ Modelo cargado correctamente")
print(f"   Features esperadas: {len(scaler.feature_names_in_)}")

# ===========================
# 2. FUNCIONES AUXILIARES (IGUAL)
# ===========================

def calcular_probabilidades_poisson(lambda_pred, rango_inferior=5, rango_superior=5):
    """Calcula probabilidades usando distribución de Poisson"""
    
    valor_central = int(round(lambda_pred))
    valores_analizar = range(
        max(0, valor_central - rango_inferior),
        valor_central + rango_superior + 1
    )
    
    probabilidades_exactas = {}
    for k in valores_analizar:
        prob = poisson.pmf(k, lambda_pred) * 100
        probabilidades_exactas[k] = prob
    
    # ✅ CORRECCIÓN: MISMAS LÍNEAS PARA OVER Y UNDER
    lines = [7.5, 8.5, 9.5, 10.5, 11.5, 12.5]
    
    probabilidades_over = {}
    for linea in lines:
        prob_over = (1 - poisson.cdf(linea, lambda_pred)) * 100
        probabilidades_over[linea] = prob_over
    
    probabilidades_under = {}
    for linea in lines:  # ✅ CAMBIO: usar la misma lista
        prob_under = poisson.cdf(linea, lambda_pred) * 100
        probabilidades_under[linea] = prob_under
    
    return {
        'exactas': probabilidades_exactas,
        'over': probabilidades_over,
        'under': probabilidades_under
    }

def clasificar_confianza(prob):
    """Clasifica la confianza según probabilidad"""
    if prob >= 66:
        return "ALTA ✅"
    elif prob >= 55:
        return "MEDIA ⚠️"
    else:
        return "BAJA ❌"

# ===========================
# 3. ✅ NUEVA FUNCIÓN GET_AVERAGE CON MÉTRICAS AVANZADAS
# ===========================

def get_average(df, is_team=False, lst_avg=None):
    """Calcula promedios de estadísticas (VERSIÓN COMPLETA)"""
    
    if len(df) == 0:
        if is_team:
            # ✅ Retornar 23 valores (métricas avanzadas)
            return (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
        return (0, 0, 0, 0, 0, 0, 0, 0, 0)
    
    if is_team:
        # ===========================
        # ESTADÍSTICAS BÁSICAS (NORMALIZADAS)
        # ===========================
        avg_cross = (df['Performance_Crs'].sum() / len(df)) - lst_avg[3]
        avg_att_3rd = (df['Touches_Att 3rd'].sum() / len(df)) - lst_avg[4]
        avg_sca = (df['SCA Types_SCA'].sum() / len(df)) - lst_avg[2]
        avg_xg = (df['Expected_xG'].sum() / len(df)) - lst_avg[1]
        
        # ✅ VARIANZA DE CORNERS
        var_ck = df['Pass Types_CK'].var() if len(df) > 1 else 0
        avg_ck = (df['Pass Types_CK'].sum() / len(df)) - lst_avg[8]
        
        avg_poss = (df['Poss'].sum() / len(df)) - 50
        avg_gf = (df['GF'].sum() / len(df)) - lst_avg[5]
        avg_ga = (df['GA'].sum() / len(df)) - lst_avg[6]
        
        # ===========================
        # MÉTRICAS OFENSIVAS AVANZADAS
        # ===========================
        total_sh = df['Standard_Sh'].sum()
        sh_accuracy = (df['Standard_SoT'].sum() / total_sh) if total_sh > 0 else 0
        xg_shot = (df['Expected_xG'].sum() / total_sh) if total_sh > 0 else 0
        
        total_touches = df['Touches_Touches'].sum()
        attacking_presence = (df['Touches_Att 3rd'].sum() / total_touches) if total_touches > 0 else 0
        
        total_poss = df['Poss'].sum()
        possession_shot = (total_sh / total_poss) if total_poss > 0 else 0
        
        # ===========================
        # MÉTRICAS DE CREACIÓN
        # ===========================
        total_passes = df['Total_Att'].sum()
        progressive_pass_ratio = (df['PrgP'].sum() / total_passes) if total_passes > 0 else 0
        final_third_involvement = (df['1/3'].sum() / total_passes) if total_passes > 0 else 0
        
        total_sca = df['SCA Types_SCA'].sum()
        assist_sca = (df['Ast'].sum() / total_sca) if total_sca > 0 else 0
        creative_efficiency = (total_sca / total_poss) if total_poss > 0 else 0
        
        # ===========================
        # MÉTRICAS DEFENSIVAS
        # ===========================
        total_tackles = df['Tackles_Tkl'].sum()
        high_press_intensity = (df['Tackles_Att 3rd'].sum() / total_tackles) if total_tackles > 0 else 0
        interception_tackle = (df['Int'].sum() / total_tackles) if total_tackles > 0 else 0
        
        total_defensive_actions = total_tackles + df['Int'].sum()
        clearance_ratio = (df['Clr'].sum() / total_defensive_actions) if total_defensive_actions > 0 else 0
        
        # ===========================
        # MÉTRICAS DE POSESIÓN
        # ===========================
        total_carries = df['Carries_Carries'].sum()
        progressive_carry_ratio = (df['Carries_PrgC'].sum() / total_carries) if total_carries > 0 else 0
        
        total_prog_passes = df['PrgP'].sum()
        carry_pass_balance = (df['Carries_PrgC'].sum() / total_prog_passes) if total_prog_passes > 0 else 0
        
        # ===========================
        # ÍNDICES COMPUESTOS
        # ===========================
        avg_gf_raw = df['GF'].mean()
        avg_xg_raw = df['Expected_xG'].mean()
        avg_sot = df['Standard_SoT'].mean()
        avg_sh = df['Standard_Sh'].mean()
        offensive_index = (avg_gf_raw + avg_xg_raw) * (avg_sot / avg_sh) if avg_sh > 0 else 0
        
        avg_prgp = df['PrgP'].mean()
        avg_prgc = df['Carries_PrgC'].mean()
        avg_poss_raw = df['Poss'].mean()
        transition_index = ((avg_prgp + avg_prgc) / avg_poss_raw) if avg_poss_raw > 0 else 0
        
        # ✅ RETORNAR 23 VALORES
        return (
            avg_ck,           # 0
            var_ck,           # 1 - ✅ NUEVO
            avg_xg,           # 2
            avg_sca,          # 3
            avg_cross,        # 4
            avg_poss,         # 5
            avg_att_3rd,      # 6
            avg_gf,           # 7
            avg_ga,           # 8
            sh_accuracy,      # 9
            xg_shot,          # 10
            attacking_presence,  # 11
            possession_shot,  # 12
            progressive_pass_ratio,  # 13
            final_third_involvement,  # 14
            assist_sca,       # 15
            creative_efficiency,  # 16
            high_press_intensity,  # 17
            interception_tackle,  # 18
            clearance_ratio,  # 19
            progressive_carry_ratio,  # 20
            carry_pass_balance,  # 21
            offensive_index,  # 22
            transition_index  # 23
        )
    
    # ===========================
    # PROMEDIOS DE LIGA (is_team=False)
    # ===========================
    avg_cross = df['Performance_Crs'].mean()
    avg_att_3rd = df['Touches_Att 3rd'].mean()
    avg_sca = df['SCA Types_SCA'].mean()
    avg_xg = df['Expected_xG'].mean()
    var_ck = df['Pass Types_CK'].var() if len(df) > 1 else 0
    avg_ck = df['Pass Types_CK'].mean()
    avg_gf = df['GF'].mean()
    avg_ga = df['GA'].mean()
    avg_sh = df['Standard_Sh'].mean() if 'Standard_Sh' in df.columns else 0
    
    return (
        var_ck,      # 0
        avg_xg,      # 1
        avg_sca,     # 2
        avg_cross,   # 3
        avg_att_3rd, # 4
        avg_gf,      # 5
        avg_ga,      # 6
        avg_sh,      # 7
        avg_ck       # 8
    )

# ===========================
# 4. ✅ NUEVA FUNCIÓN GET_TEAM_PPP
# ===========================

def get_points_from_result(result):
    """Convierte resultado (W/D/L) a puntos"""
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

def get_team_ppp(df, team, season, round_num, league=None):
    """Calcula puntos por partido (PPP) de un equipo"""
    team_matches = df[
        (df['team'] == team) & 
        (df['season'] == season) & 
        (df['round'] < round_num)
    ]
    
    if league is not None:
        team_matches = team_matches[team_matches['league'] == league]
    
    if len(team_matches) == 0:
        return 0.0
    
    total_points = team_matches['result'].apply(get_points_from_result).sum()
    ppp = total_points / len(team_matches)
    
    return ppp

def get_ppp_difference(df, local, away, season, round_num, league=None):
    """Calcula diferencia de PPP entre local y visitante"""
    local_ppp = get_team_ppp(df, local, season, round_num, league)
    away_ppp = get_team_ppp(df, away, season, round_num, league)
    return local_ppp - away_ppp


# ===========================
# 5. ✅ FUNCIÓN PRINCIPAL DE PREDICCIÓN (ACTUALIZADA)
# ===========================

def predecir_corners(local, visitante, jornada, temporada="2526", league_code="ESP"):
    """
    Predice corners totales con análisis completo para apuestas
    
    Args:
        local: Equipo local
        visitante: Equipo visitante
        jornada: Número de jornada
        temporada: Temporada (formato "2526")
        league_code: Código de liga ("ESP", "GER", "FRA", "ITA", "NED")
    """
    
    print(f"\n{'='*80}")
    print(f"🏟️  {local} vs {visitante}")
    print(f"📅 Temporada {temporada} | Jornada {jornada} | Liga: {league_code}")
    print(f"{'='*80}")
    
    if jornada < 5:
        return {
            "error": "❌ Se necesitan al menos 5 jornadas previas",
            "prediccion": None
        }
    
    try:
        # ===========================
        # EXTRAER FEATURES (igual que antes)
        # ===========================
        
        lst_avg = get_average(
            df_database[
                (df_database['season'] == temporada) & 
                (df_database['round'] < jornada) &
                (df_database['league'] == league_code)
            ],
            is_team=False
        )
        
        (team1_home, team1_away, team1_opp_home, team1_opp_away,
         team2_home, team2_away, team2_opp_home, team2_opp_away) = get_dataframes(
            df_database, temporada, jornada, local, visitante, league=league_code
        )
        
        index = lst_years.index(temporada)
        result = lst_years[:index+1]
        team1_h2h, team2_h2h = get_head_2_head(
            df_database, local, visitante, seasons=result, league=league_code
        )
        
        local_ppp = get_team_ppp(df_database, local, temporada, jornada, league=league_code)
        away_ppp = get_team_ppp(df_database, visitante, temporada, jornada, league=league_code)
        ppp_diff = local_ppp - away_ppp
        
        # ===========================
        # CONSTRUIR DICCIONARIO DE FEATURES (igual que antes)
        # ===========================
        
        def create_line(df, is_form=True, is_team=False, use_advanced=True):
            if is_form:
                df = df[-6:]
            if use_advanced:
                return get_average(df, is_team, lst_avg)
            else:
                result = get_average(df, is_team, lst_avg)
                return result[:9]
        
        dic_features = {}
        
        dic_features['ppp_local'] = (local_ppp,)
        dic_features['ppp_away'] = (away_ppp,)
        dic_features['ppp_difference'] = (ppp_diff,)
        
        dic_features['lst_team1_home_form'] = create_line(team1_home, True, True, use_advanced=True)
        dic_features['lst_team1_home_general'] = create_line(team1_home, False, True, use_advanced=True)
        dic_features['lst_team1_away_form'] = create_line(team1_away, True, True, use_advanced=True)
        dic_features['lst_team1_away_general'] = create_line(team1_away, False, True, use_advanced=True)
        
        dic_features['lst_team2_home_form'] = create_line(team2_home, True, True, use_advanced=True)
        dic_features['lst_team2_home_general'] = create_line(team2_home, False, True, use_advanced=True)
        dic_features['lst_team2_away_form'] = create_line(team2_away, True, True, use_advanced=True)
        dic_features['lst_team2_away_general'] = create_line(team2_away, False, True, use_advanced=True)
        
        dic_features['lst_team1_h2h'] = create_line(team1_h2h, False, True, use_advanced=True)
        dic_features['lst_team2_h2h'] = create_line(team2_h2h, False, True, use_advanced=True)
        
        dic_features['lst_team1_opp_away'] = create_line(team1_opp_away, False, True, use_advanced=False)
        dic_features['lst_team2_opp_home'] = create_line(team2_opp_home, False, True, use_advanced=False)
        
        league_dummies = {
            'league_ESP': 1 if league_code == 'ESP' else 0,
            'league_GER': 1 if league_code == 'GER' else 0,
            'league_FRA': 1 if league_code == 'FRA' else 0,
            'league_ITA': 1 if league_code == 'ITA' else 0,
            'league_NED': 1 if league_code == 'NED' else 0
        }
        
        for key, value in league_dummies.items():
            dic_features[key] = (value,)
        
        # ===========================
        # CONSTRUIR VECTOR DE FEATURES
        # ===========================
        
        lst_base_advanced = [
            "avg_ck", "var_ck", "xg", "sca", "cross", "poss", "att_3rd", "gf", "ga",
            "sh_accuracy", "xg_shot", "attacking_presence", "possession_shot",
            "progressive_pass_ratio", "final_third_involvement", "assist_sca", "creative_efficiency",
            "high_press_intensity", "interception_tackle", "clearance_ratio",
            "progressive_carry_ratio", "carry_pass_balance", "offensive_index", "transition_index"
        ]
        
        lst_base_original = [
            "var_ck", "xg", "sca", "cross", "poss", "att_3rd", "gf", "ga", "avg_ck"
        ]
        
        lst_features_values = []
        lst_features_names = []
        
        for key in dic_features:
            lst_features_values.extend(list(dic_features[key]))
            
            if key in ['ppp_local', 'ppp_away', 'ppp_difference']:
                lst_features_names.append(key)
            elif key.startswith('league_'):
                lst_features_names.append(key)
            elif key in ['lst_team1_opp_away', 'lst_team2_opp_home']:
                lst_features_names.extend([f"{key}_{col}" for col in lst_base_original])
            else:
                lst_features_names.extend([f"{key}_{col}" for col in lst_base_advanced])
        
        df_input = pd.DataFrame([lst_features_values], columns=lst_features_names)
        
        expected_features = scaler.feature_names_in_
        
        if len(df_input.columns) != len(expected_features):
            print(f"\n⚠️ ERROR: Número de features no coincide")
            print(f"   Esperadas: {len(expected_features)}")
            print(f"   Recibidas: {len(df_input.columns)}")
            return {"error": "Desajuste de features", "prediccion": None}
        
        df_input = df_input[expected_features]
        
        X_input_scaled = pd.DataFrame(
            scaler.transform(df_input), 
            columns=df_input.columns
        )
        
        # ===========================
        # PREDICCIÓN
        # ===========================
        
        prediccion = xgb_model.predict(X_input_scaled)[0]
        
        # ===========================
        # ✅ ANÁLISIS PROBABILÍSTICO CON POISSON
        # ===========================
        
        analisis = calcular_probabilidades_poisson(prediccion, rango_inferior=5, rango_superior=5)
        
        # ===========================
        # ESTADÍSTICAS DETALLADAS
        # ===========================
        
        local_ck_home = team1_home['Pass Types_CK'].mean() if len(team1_home) > 0 else 0
        local_xg_home = team1_home['Expected_xG'].mean() if len(team1_home) > 0 else 0
        local_poss_home = team1_home['Poss'].mean() if len(team1_home) > 0 else 0
        
        away_ck_away = team2_away['Pass Types_CK'].mean() if len(team2_away) > 0 else 0
        away_xg_away = team2_away['Expected_xG'].mean() if len(team2_away) > 0 else 0
        away_poss_away = team2_away['Poss'].mean() if len(team2_away) > 0 else 0
        
        local_ck_received = team1_opp_home['Pass Types_CK'].mean() if len(team1_opp_home) > 0 else 0
        away_ck_received = team2_opp_away['Pass Types_CK'].mean() if len(team2_opp_away) > 0 else 0
        
        partido_ck_esperado = local_ck_home + away_ck_away
        
        h2h_ck_local = team1_h2h['Pass Types_CK'].mean() if len(team1_h2h) > 0 else 0
        h2h_ck_away = team2_h2h['Pass Types_CK'].mean() if len(team2_h2h) > 0 else 0
        h2h_total = h2h_ck_local + h2h_ck_away
        
        # ===========================
        # ✅ MOSTRAR RESULTADOS CON PROBABILIDADES
        # ===========================
        
        print(f"\n🎲 PREDICCIÓN MODELO: {prediccion:.2f} corners totales")
        print(f"   PPP: {local} ({local_ppp:.2f}) vs {visitante} ({away_ppp:.2f}) | Diff: {ppp_diff:+.2f}")
        
        print(f"\n📊 ESTADÍSTICAS HISTÓRICAS:")
        print(f"   {local} (Casa): {local_ck_home:.1f} CK/partido | xG: {local_xg_home:.2f} | Poss: {local_poss_home:.1f}%")
        print(f"   {visitante} (Fuera): {away_ck_away:.1f} CK/partido | xG: {away_xg_away:.2f} | Poss: {away_poss_away:.1f}%")
        print(f"   Corners recibidos: {local} ({local_ck_received:.1f}) | {visitante} ({away_ck_received:.1f})")
        print(f"   Total esperado (suma): {partido_ck_esperado:.1f} corners")
        
        if len(team1_h2h) > 0 or len(team2_h2h) > 0:
            print(f"\n🔄 HEAD TO HEAD (últimos {max(len(team1_h2h), len(team2_h2h))} partidos):")
            print(f"   {local}: {h2h_ck_local:.1f} CK/partido")
            print(f"   {visitante}: {h2h_ck_away:.1f} CK/partido")
            print(f"   Promedio total: {h2h_total:.1f} corners")
        
        # ===========================
        # ✅ MOSTRAR PROBABILIDADES EXACTAS
        # ===========================
        
        valor_mas_probable = max(analisis['exactas'].items(), key=lambda x: x[1])
        
        print(f"\n📈 PROBABILIDADES EXACTAS (Poisson):")
        for k in sorted(analisis['exactas'].keys()):
            prob = analisis['exactas'][k]
            bar = '█' * int(prob / 2)
            marca = ' ⭐' if k == valor_mas_probable[0] else ''
            print(f"   {k:2d} corners: {prob:5.2f}% {bar}{marca}")
        
        print(f"\n✅ Valor más probable: {valor_mas_probable[0]} corners ({valor_mas_probable[1]:.2f}%)")
        
        # ✅ RANGO DE 80% CONFIANZA
        probs_sorted = sorted(analisis['exactas'].items(), key=lambda x: x[1], reverse=True)
        cumsum = 0
        rango_80 = []
        for val, prob in probs_sorted:
            cumsum += prob
            rango_80.append(val)
            if cumsum >= 80:
                break
        
        print(f"📊 Rango 80% confianza: {min(rango_80)}-{max(rango_80)} corners")
        
        # ===========================
        # ✅ MOSTRAR OVER/UNDER CON CUOTAS IMPLÍCITAS
        # ===========================
        
        print(f"\n🎯 ANÁLISIS OVER/UNDER:")
        print(f"{'Línea':<10} {'Prob Over':<12} {'Cuota Impl':<12} {'Confianza':<15} {'Prob Under':<12} {'Cuota Impl':<12}")
        print("-" * 85)
        
        for linea in [7.5, 8.5, 9.5, 10.5, 11.5, 12.5]:
            prob_over = analisis['over'][linea]
            prob_under = analisis['under'][linea]
            
            # Cuotas implícitas (inverso de probabilidad en decimal)
            cuota_impl_over = 100 / prob_over if prob_over > 0 else 999
            cuota_impl_under = 100 / prob_under if prob_under > 0 else 999
            
            conf_over = clasificar_confianza(prob_over)
            
            print(f"O/U {linea:<5} {prob_over:6.2f}%     @{cuota_impl_over:5.2f}      {conf_over:<15} {prob_under:6.2f}%     @{cuota_impl_under:5.2f}")
        
        # ===========================
        # ✅ RECOMENDACIONES CON CUOTAS
        # ===========================
        
        print(f"\n💡 RECOMENDACIONES DE APUESTA:")
        
        mejores_over = [(l, p) for l, p in analisis['over'].items() if p >= 55]
        mejores_under = [(l, p) for l, p in analisis['under'].items() if p >= 55]
        
        if mejores_over:
            print(f"\n✅ OVER con confianza MEDIA/ALTA:")
            for linea, prob in sorted(mejores_over, key=lambda x: x[1], reverse=True):
                cuota_impl = 100 / prob
                conf = clasificar_confianza(prob)
                print(f"   • Over {linea}: {prob:.2f}% (Cuota justa: @{cuota_impl:.2f}) - {conf}")
        
        if mejores_under:
            print(f"\n✅ UNDER con confianza MEDIA/ALTA:")
            for linea, prob in sorted(mejores_under, key=lambda x: x[1], reverse=True):
                cuota_impl = 100 / prob
                conf = clasificar_confianza(prob)
                print(f"   • Under {linea}: {prob:.2f}% (Cuota justa: @{cuota_impl:.2f}) - {conf}")
        
        if not mejores_over and not mejores_under:
            print(f"   ⚠️ No hay apuestas con confianza MEDIA o superior")
        
        # ===========================
        # ✅ ANÁLISIS DE RIESGO
        # ===========================
        
        df_varianza_temp = analizar_fiabilidad_equipos(df_database, temporada=temporada, min_partidos=3)
        riesgo = obtener_fiabilidad_partido(local, visitante, df_varianza_temp)

        print(f"\n⚠️ ANÁLISIS DE RIESGO:")
        print(f"   Local ({local}): {riesgo['nivel_local']} (CV: {riesgo['cv_local']:.1f}%)")
        print(f"   Away ({visitante}): {riesgo['nivel_away']} (CV: {riesgo['cv_away']:.1f}%)")
        print(f"   🎲 FIABILIDAD PARTIDO: {riesgo['fiabilidad']} (Score: {riesgo['score_promedio']:.1f})")
        print(f"   💡 {riesgo['mensaje']}")
        
        # ===========================
        # RETORNAR DICCIONARIO COMPLETO
        # ===========================
        
        return {
            "prediccion": round(prediccion, 2),
            "local": local,
            "visitante": visitante,
            "ppp_local": local_ppp,
            "ppp_away": away_ppp,
            "ppp_diff": ppp_diff,
            "riesgo": riesgo,
            "stats": {
                "local_ck": local_ck_home,
                "away_ck": away_ck_away,
                "local_ck_received": local_ck_received,
                "away_ck_received": away_ck_received,
                "h2h_total": h2h_total,
                "partido_esperado": partido_ck_esperado
            },
            "probabilidades_exactas": analisis['exactas'],
            "probabilidades_over": analisis['over'],
            "probabilidades_under": analisis['under'],
            "valor_mas_probable": valor_mas_probable[0],
            "prob_mas_probable": valor_mas_probable[1],
            "rango_80": (min(rango_80), max(rango_80))
        }
        
    except Exception as e:
        print(f"\n❌ ERROR: {str(e)}")
        import traceback
        traceback.print_exc()
        return {"error": str(e), "prediccion": None}

print("\n✅ Sistema listo con probabilidades Poisson y cuotas implícitas")


🎯 SISTEMA DE PREDICCIÓN DE CORNERS PARA APUESTAS (VERSIÓN AVANZADA)
✅ Modelo cargado correctamente
   Features esperadas: 266

✅ Sistema listo con probabilidades Poisson y cuotas implícitas


In [23]:
# ===========================
# SISTEMA AVANZADO DE DETECCIÓN DE PARTIDOS APOSTABLES
# ===========================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

def analizar_fiabilidad_equipos(df_database, temporada="2526", min_partidos=5):
    """
    Análisis completo de fiabilidad para apuestas de corners
    No solo varianza, sino consistencia, tendencias y patrones
    """
    
    df_temp = df_database[df_database['season'] == temporada].copy()
    resultados = []
    equipos = pd.concat([df_temp['team'], df_temp['opponent']]).unique()
    
    for equipo in equipos:
        # Partidos del equipo
        partidos_equipo = df_temp[df_temp['team'] == equipo]
        
        if len(partidos_equipo) < min_partidos:
            continue
        
        ck_sacados = partidos_equipo['Pass Types_CK'].values
        
        # ===========================
        # 1. MÉTRICAS DE VARIABILIDAD
        # ===========================
        media = ck_sacados.mean()
        std = ck_sacados.std()
        cv = (std / media * 100) if media > 0 else 0
        
        # ===========================
        # 2. MÉTRICAS DE CONSISTENCIA
        # ===========================
        
        # 2.1 Porcentaje de partidos cerca de la media (±2 corners)
        cerca_media = np.sum(np.abs(ck_sacados - media) <= 2) / len(ck_sacados) * 100
        
        # 2.2 Rachas (detectar equipos con "explosiones" de corners)
        cambios_bruscos = np.sum(np.abs(np.diff(ck_sacados)) > 4)
        pct_cambios_bruscos = cambios_bruscos / (len(ck_sacados) - 1) * 100
        
        # 2.3 Cuartiles (Q1, Q2=mediana, Q3)
        q1, q2, q3 = np.percentile(ck_sacados, [25, 50, 75])
        iqr = q3 - q1  # Rango intercuartílico (más robusto que std)
        
        # ===========================
        # 3. MÉTRICAS DE TENDENCIA
        # ===========================
        
        # 3.1 Tendencia lineal (¿mejora/empeora con el tiempo?)
        jornadas = np.arange(len(ck_sacados))
        slope, intercept, r_value, p_value, std_err = stats.linregress(jornadas, ck_sacados)
        
        # 3.2 Autocorrelación (¿resultado actual predice el siguiente?)
        if len(ck_sacados) > 2:
            autocorr = np.corrcoef(ck_sacados[:-1], ck_sacados[1:])[0, 1]
        else:
            autocorr = 0
        
        # ===========================
        # 4. MÉTRICAS DE OUTLIERS
        # ===========================
        
        # 4.1 Detección de valores atípicos (método IQR)
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = np.sum((ck_sacados < lower_bound) | (ck_sacados > upper_bound))
        pct_outliers = outliers / len(ck_sacados) * 100
        
        # 4.2 Z-score máximo
        z_scores = np.abs(stats.zscore(ck_sacados))
        max_z = z_scores.max()
        
        # ===========================
        # 5. MÉTRICAS DE RANGO
        # ===========================
        
        rango = ck_sacados.max() - ck_sacados.min()
        rango_normalizado = rango / media if media > 0 else 0
        
        # ===========================
        # 6. SCORE GLOBAL DE FIABILIDAD
        # ===========================
        
        # Penalizaciones (0-100, menor = peor)
        score_cv = max(0, 100 - cv * 2)  # CV alto = mala
        score_consistencia = cerca_media  # Más cerca de media = mejor
        score_cambios = max(0, 100 - pct_cambios_bruscos * 2)  # Cambios bruscos = malo
        score_outliers = max(0, 100 - pct_outliers * 3)  # Outliers = malo
        score_iqr = max(0, 100 - iqr * 10)  # IQR grande = malo
        
        # Score final (promedio ponderado)
        score_fiabilidad = (
            score_cv * 0.25 +
            score_consistencia * 0.30 +
            score_cambios * 0.20 +
            score_outliers * 0.15 +
            score_iqr * 0.10
        )
        
        # ===========================
        # 7. CLASIFICACIÓN MULTI-CRITERIO
        # ===========================
        
        # Clasificación basada en score
        if score_fiabilidad >= 70:
            nivel = "EXCELENTE ⭐⭐⭐"
            color = "#27ae60"
        elif score_fiabilidad >= 55:
            nivel = "BUENO ✅"
            color = "#2ecc71"
        elif score_fiabilidad >= 40:
            nivel = "ACEPTABLE 🟡"
            color = "#f39c12"
        elif score_fiabilidad >= 25:
            nivel = "REGULAR ⚠️"
            color = "#e67e22"
        else:
            nivel = "EVITAR ⛔"
            color = "#e74c3c"
        
        resultados.append({
            'Equipo': equipo,
            'Partidos': len(ck_sacados),
            
            # Estadísticas básicas
            'Media_CK': round(media, 2),
            'Mediana_CK': round(q2, 2),
            'Std_CK': round(std, 2),
            'CV_%': round(cv, 1),
            
            # Consistencia
            'Pct_Cerca_Media': round(cerca_media, 1),
            'Cambios_Bruscos_%': round(pct_cambios_bruscos, 1),
            'IQR': round(iqr, 2),
            
            # Rango
            'Rango': int(rango),
            'Rango_Norm': round(rango_normalizado, 2),
            'Min': int(ck_sacados.min()),
            'Max': int(ck_sacados.max()),
            
            # Outliers
            'Outliers': int(outliers),
            'Pct_Outliers': round(pct_outliers, 1),
            'Max_ZScore': round(max_z, 2),
            
            # Tendencia
            'Tendencia_Slope': round(slope, 3),
            'Autocorr': round(autocorr, 3),
            
            # Score y clasificación
            'Score_Fiabilidad': round(score_fiabilidad, 1),
            'Nivel': nivel,
            'Color': color
        })
    
    df_resultado = pd.DataFrame(resultados)

    print(df_resultado.head(10))
    df_resultado = df_resultado.sort_values('Score_Fiabilidad', ascending=False)
    
    return df_resultado


def mostrar_analisis_fiabilidad(df_analisis, top_n=10):
    """
    Muestra el análisis completo de fiabilidad
    """
    
    print("\n" + "=" * 120)
    print("🎯 ANÁLISIS DE FIABILIDAD PARA APUESTAS - CORNERS")
    print("=" * 120)
    
    # TOP EQUIPOS FIABLES
    print(f"\n⭐ TOP {top_n} EQUIPOS MÁS FIABLES")
    print("-" * 120)
    
    top_fiables = df_analisis.head(top_n)
    
    for idx, row in top_fiables.iterrows():
        print(f"\n{row['Equipo']:25s} | {row['Nivel']:20s} | Score: {row['Score_Fiabilidad']:.1f}")
        print(f"  📊 Media: {row['Media_CK']:.1f} | Mediana: {row['Mediana_CK']:.1f} | CV: {row['CV_%']:.1f}%")
        print(f"  ✅ {row['Pct_Cerca_Media']:.1f}% cerca de media | IQR: {row['IQR']:.1f}")
        print(f"  ⚠️ Cambios bruscos: {row['Cambios_Bruscos_%']:.1f}% | Outliers: {row['Pct_Outliers']:.1f}%")
        print(f"  📈 Rango: {row['Min']}-{row['Max']} ({row['Rango']} corners)")
    
    # TOP EQUIPOS NO FIABLES
    print(f"\n\n⛔ TOP {top_n} EQUIPOS MENOS FIABLES")
    print("-" * 120)
    
    top_no_fiables = df_analisis.tail(top_n)
    
    for idx, row in top_no_fiables.iterrows():
        print(f"\n{row['Equipo']:25s} | {row['Nivel']:20s} | Score: {row['Score_Fiabilidad']:.1f}")
        print(f"  📊 Media: {row['Media_CK']:.1f} | Mediana: {row['Mediana_CK']:.1f} | CV: {row['CV_%']:.1f}%")
        print(f"  ❌ Solo {row['Pct_Cerca_Media']:.1f}% cerca de media | IQR: {row['IQR']:.1f}")
        print(f"  ⚠️ Cambios bruscos: {row['Cambios_Bruscos_%']:.1f}% | Outliers: {row['Pct_Outliers']:.1f}%")
    
    # ESTADÍSTICAS GENERALES
    print(f"\n\n📊 DISTRIBUCIÓN POR NIVEL DE FIABILIDAD")
    print("-" * 120)
    print(df_analisis['Nivel'].value_counts())
    
    print(f"\n📈 ESTADÍSTICAS DE SCORE:")
    print(f"  Media: {df_analisis['Score_Fiabilidad'].mean():.1f}")
    print(f"  Mediana: {df_analisis['Score_Fiabilidad'].median():.1f}")
    print(f"  Score máximo: {df_analisis['Score_Fiabilidad'].max():.1f}")
    print(f"  Score mínimo: {df_analisis['Score_Fiabilidad'].min():.1f}")


def obtener_fiabilidad_partido(local, visitante, df_analisis):
    """
    Evalúa la fiabilidad de un partido específico
    """
    
    datos_local = df_analisis[df_analisis['Equipo'] == local]
    datos_away = df_analisis[df_analisis['Equipo'] == visitante]
    
    if datos_local.empty or datos_away.empty:
        return {
            'fiabilidad': 'DESCONOCIDO',
            'score': 0,
            'mensaje': '⚠️ Datos insuficientes'
        }
    
    score_local = datos_local['Score_Fiabilidad'].values[0]
    score_away = datos_away['Score_Fiabilidad'].values[0]
    score_promedio = (score_local + score_away) / 2
    
    # Clasificación del partido
    if score_promedio >= 65:
        fiabilidad = "MUY ALTA ⭐⭐⭐"
        mensaje = "✅ EXCELENTE PARTIDO PARA APOSTAR"
    elif score_promedio >= 50:
        fiabilidad = "ALTA ✅"
        mensaje = "✅ BUEN PARTIDO PARA APOSTAR"
    elif score_promedio >= 35:
        fiabilidad = "MEDIA 🟡"
        mensaje = "🟡 APOSTAR CON PRECAUCIÓN"
    else:
        fiabilidad = "BAJA ⛔"
        mensaje = "⛔ EVITAR APUESTA"
    
    return {
        'fiabilidad': fiabilidad,
        'score_local': score_local,
        'score_away': score_away,
        'score_promedio': score_promedio,
        'nivel_local': datos_local['Nivel'].values[0],
        'nivel_away': datos_away['Nivel'].values[0],
        'mensaje': mensaje,
        
        # Datos adicionales útiles
        'cv_local': datos_local['CV_%'].values[0],
        'cv_away': datos_away['CV_%'].values[0],
        'consistencia_local': datos_local['Pct_Cerca_Media'].values[0],
        'consistencia_away': datos_away['Pct_Cerca_Media'].values[0]
    }


# ===========================
# EJECUTAR ANÁLISIS
# ===========================

df_fiabilidad = analizar_fiabilidad_equipos(df_database, temporada="2526", min_partidos=5)

mostrar_analisis_fiabilidad(df_fiabilidad, top_n=10)

# Guardar resultados
df_fiabilidad.to_csv('analisis_fiabilidad_equipos.csv', index=False)
print("\n💾 Resultados guardados en 'analisis_fiabilidad_equipos.csv'")

# ===========================
# EJEMPLO DE USO
# ===========================

print("\n" + "=" * 120)
print("🎯 EJEMPLO: ANÁLISIS DE FIABILIDAD DE PARTIDO")
print("=" * 120)

fiabilidad = obtener_fiabilidad_partido("Barcelona", "Elche", df_fiabilidad)

print(f"\n🏟️ PARTIDO: Marseille vs Angers")
print(f"{'='*80}")
print(f"Local (Marseille): {fiabilidad['nivel_local']} (Score: {fiabilidad['score_local']:.1f})")
print(f"  • CV: {fiabilidad['cv_local']:.1f}% | Consistencia: {fiabilidad['consistencia_local']:.1f}%")
print(f"\nAway (Angers): {fiabilidad['nivel_away']} (Score: {fiabilidad['score_away']:.1f})")
print(f"  • CV: {fiabilidad['cv_away']:.1f}% | Consistencia: {fiabilidad['consistencia_away']:.1f}%")
print(f"\n🎲 FIABILIDAD DEL PARTIDO: {fiabilidad['fiabilidad']}")
print(f"Score Promedio: {fiabilidad['score_promedio']:.1f}/100")
print(f"\n💡 {fiabilidad['mensaje']}")
print("=" * 80)

           Equipo  Partidos  Media_CK  Mediana_CK  Std_CK  CV_%  \
0         Arsenal        10      7.30         8.0    3.10  42.5   
1     Aston Villa        10      5.30         5.0    2.45  46.3   
2     Bournemouth         9      5.56         5.0    1.57  28.3   
3       Brentford        10      5.30         5.0    2.53  47.8   
4        Brighton        10      4.80         4.0    2.18  45.5   
5         Burnley        10      2.60         2.5    1.50  57.6   
6         Chelsea        10      6.10         5.5    2.34  38.4   
7  Crystal Palace        10      3.80         3.5    2.27  59.8   
8         Everton         9      4.11         3.0    2.96  72.0   
9          Fulham        10      5.10         3.5    2.77  54.4   

   Pct_Cerca_Media  Cambios_Bruscos_%   IQR  Rango  ...  Min  Max  Outliers  \
0             40.0               22.2  5.00     10  ...    2   12         0   
1             50.0               22.2  2.75      8  ...    2   10         0   
2             77.8       

In [24]:
def get_ck(df, season, round_num, local, away, league=None):
    """Obtiene corners totales de un partido específico"""
    season_round = (df['season'] == season) & (df['round'] == round_num)
    
    if league is not None:
        season_round = season_round & (df['league'] == league)
    
    df = df[season_round]
    
    df_local = df[df['team'] == local]
    df_away = df[df['team'] == away]
    
    total_ck = df_local["Pass Types_CK"].sum() + df_away["Pass Types_CK"].sum()
    
    return total_ck

def get_dataframes(df, season, round_num, local, away, league=None):
    """Retorna 8 DataFrames filtrados por equipo, venue y liga"""
    
    season_round = (df['season'] == season) & (df['round'] < round_num)
    
    if league is not None:
        season_round = season_round & (df['league'] == league)
    
    def filter_and_split(team_filter):
        filtered = df[season_round & team_filter].copy()
        home = filtered[filtered['venue'] == "Home"]
        away = filtered[filtered['venue'] == "Away"]
        return home, away
    
    local_home, local_away = filter_and_split(df['team'] == local)
    local_opp_home, local_opp_away = filter_and_split(df['opponent'] == local)
    
    away_home, away_away = filter_and_split(df['team'] == away)
    away_opp_home, away_opp_away = filter_and_split(df['opponent'] == away)
    
    return (local_home, local_away, local_opp_home, local_opp_away,
            away_home, away_away, away_opp_home, away_opp_away)

def get_head_2_head(df, local, away, seasons=None, league=None):
    """Obtiene últimos 3 enfrentamientos directos"""
    if seasons is None:
        seasons = []
    
    df_filtered = df[df['season'].isin(seasons)] if seasons else df
    
    if league is not None:
        df_filtered = df_filtered[df_filtered['league'] == league]
    
    local_h2h = df_filtered[(df_filtered['team'] == local) & (df_filtered['opponent'] == away)]
    away_h2h = df_filtered[(df_filtered['team'] == away) & (df_filtered['opponent'] == local)]
    
    if len(local_h2h) < 4:
        return local_h2h.tail(2), away_h2h.tail(2)
    
    return local_h2h.tail(3), away_h2h.tail(3)

def get_points_from_result(result):
    """Convierte resultado (W/D/L) a puntos"""
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

# ✅ NUEVA FUNCIÓN: Calcular PPP (Puntos Por Partido)
def get_team_ppp(df, team, season, round_num, league=None):
    """
    Calcula puntos por partido (PPP) de un equipo
    
    Args:
        df: DataFrame completo
        team: Nombre del equipo
        season: Temporada
        round_num: Número de jornada (NO incluye esta jornada)
        league: Código de liga (opcional)
    
    Returns:
        float: Puntos por partido (0-3)
    """
    team_matches = df[
        (df['team'] == team) & 
        (df['season'] == season) & 
        (df['round'] < round_num)
    ]
    
    if league is not None:
        team_matches = team_matches[team_matches['league'] == league]
    
    if len(team_matches) == 0:
        return 0.0
    
    total_points = team_matches['result'].apply(get_points_from_result).sum()
    ppp = total_points / len(team_matches)
    
    return ppp

# ✅ NUEVA FUNCIÓN: Calcular diferencia de PPP
def get_ppp_difference(df, local, away, season, round_num, league=None):
    """
    Calcula la diferencia de puntos por partido entre local y visitante
    
    Args:
        df: DataFrame completo
        local: Equipo local
        away: Equipo visitante
        season: Temporada
        round_num: Jornada actual
        league: Código de liga (opcional)
    
    Returns:
        float: Diferencia de PPP (local - away)
    """
    local_ppp = get_team_ppp(df, local, season, round_num, league)
    away_ppp = get_team_ppp(df, away, season, round_num, league)
    
    return local_ppp - away_ppp

def get_average(df, is_team=False, lst_avg=None):
    """Calcula promedios de estadísticas"""
    
    if len(df) == 0:
        # Retornar valores por defecto si el DataFrame está vacío
        if is_team:
            return (0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
        return (0, 0, 0, 0, 0, 0, 0, 0)
    
    if is_team:
        # ===========================
        # ESTADÍSTICAS BÁSICAS (NORMALIZADAS)
        # ===========================
        avg_cross = (df['Performance_Crs'].sum() / len(df)) - lst_avg[3]
        avg_att_3rd = (df['Touches_Att 3rd'].sum() / len(df)) - lst_avg[4]
        avg_sca = (df['SCA Types_SCA'].sum() / len(df)) - lst_avg[2]
        avg_xg = (df['Expected_xG'].sum() / len(df)) - lst_avg[1]
        
        # ✅ CAMBIO: VARIANZA EN VEZ DE PROMEDIO DE CK
        var_ck = df['Pass Types_CK'].var() if len(df) > 1 else 0
        avg_ck = (df['Pass Types_CK'].sum() / len(df)) - lst_avg[8]
        
        avg_poss = (df['Poss'].sum() / len(df)) - 50
        avg_gf = (df['GF'].sum() / len(df)) - lst_avg[5]
        avg_ga = (df['GA'].sum() / len(df)) - lst_avg[6]
        
        # ===========================
        # MÉTRICAS OFENSIVAS AVANZADAS
        # ===========================
        
        # Precisión de tiros
        total_sh = df['Standard_Sh'].sum()
        sh_accuracy = (df['Standard_SoT'].sum() / total_sh) if total_sh > 0 else 0
        
        # Eficiencia xG por tiro
        xg_shot = (df['Expected_xG'].sum() / total_sh) if total_sh > 0 else 0
        
        # Presencia atacante (% toques en área rival)
        total_touches = df['Touches_Touches'].sum()
        attacking_presence = (df['Touches_Att 3rd'].sum() / total_touches) if total_touches > 0 else 0
        
        # Tiros por posesión
        total_poss = df['Poss'].sum()
        possession_shot = (total_sh / total_poss) if total_poss > 0 else 0
        
        # Distancia promedio de tiros
        standard_dist = df['Standard_Dist'].mean() if 'Standard_Dist' in df.columns else 0
        
        # ===========================
        # MÉTRICAS DE CREACIÓN
        # ===========================
        
        # Ratio de pases progresivos
        total_passes = df['Total_Att'].sum()
        progressive_pass_ratio = (df['PrgP'].sum() / total_passes) if total_passes > 0 else 0
        
        # Participación en último tercio
        final_third_passes = df['1/3'].sum()
        final_third_involvement = (final_third_passes / total_passes) if total_passes > 0 else 0
        
        # Ratio de pases largos
        long_ball_ratio = (df['Long_Att'].sum() / total_passes) if total_passes > 0 else 0
        
        # Asistencias por SCA
        total_sca = df['SCA Types_SCA'].sum()
        assist_sca = (df['Ast'].sum() / total_sca) if total_sca > 0 else 0
        
        # Dependencia de centros
        cross_dependency = (df['Performance_Crs'].sum() / total_passes) if total_passes > 0 else 0
        
        # Eficiencia creativa
        creative_efficiency = (total_sca / total_poss) if total_poss > 0 else 0
        
        # ===========================
        # MÉTRICAS DEFENSIVAS
        # ===========================
        
        # Intensidad de presión alta
        total_tackles = df['Tackles_Tkl'].sum()
        high_press_intensity = (df['Tackles_Att 3rd'].sum() / total_tackles) if total_tackles > 0 else 0
        
        # Ratio intercepciones/tackles
        interception_tackle = (df['Int'].sum() / total_tackles) if total_tackles > 0 else 0
        
        # Ratio bloqueos/tackles
        blocks_tackle = (df['Blocks_Blocks'].sum() / total_tackles) if total_tackles > 0 else 0
        
        # Ratio de despejes
        total_defensive_actions = total_tackles + df['Int'].sum()
        clearance_ratio = (df['Clr'].sum() / total_defensive_actions) if total_defensive_actions > 0 else 0
        
        # ===========================
        # MÉTRICAS DE PORTERÍA
        # ===========================
        
        # Rendimiento del portero normalizado
        avg_save_pct = df['Performance_Save%'].mean() if 'Performance_Save%' in df.columns else 0
        avg_xg_against = df['Expected_xG'].mean() if len(df) > 0 else 1
        performance_save = (avg_save_pct / (1 / avg_xg_against)) if avg_xg_against > 0 else 0
        
        # ===========================
        # MÉTRICAS DE POSESIÓN
        # ===========================
        
        # Ratio de conducciones progresivas
        total_carries = df['Carries_Carries'].sum()
        progressive_carry_ratio = (df['Carries_PrgC'].sum() / total_carries) if total_carries > 0 else 0
        
        # Ratio de conducciones al área
        penalty_carry_ratio = (df['Carries_CPA'].sum() / total_carries) if total_carries > 0 else 0
        
        # Balance conducción/pase progresivo
        total_prog_passes = df['PrgP'].sum()
        carry_pass_balance = (df['Carries_PrgC'].sum() / total_prog_passes) if total_prog_passes > 0 else 0
        
        # ===========================
        # ÍNDICES COMPUESTOS
        # ===========================
        
        # Índice ofensivo
        avg_gf_raw = df['GF'].mean()
        avg_xg_raw = df['Expected_xG'].mean()
        avg_sot = df['Standard_SoT'].mean()
        avg_sh = df['Standard_Sh'].mean()
        offensive_index = (avg_gf_raw + avg_xg_raw) * (avg_sot / avg_sh) if avg_sh > 0 else 0
        
        # Índice defensivo
        avg_int = df['Int'].mean()
        avg_tkl = df['Tackles_Tkl'].mean()
        avg_clr = df['Clr'].mean()
        defensive_index = avg_save_pct * (avg_int / (avg_tkl + avg_clr)) if (avg_tkl + avg_clr) > 0 else 0
        
        # Índice de control de posesión
        avg_touches_att = df['Touches_Att 3rd'].mean()
        avg_carries_third = df['Carries_1/3'].mean() if 'Carries_1/3' in df.columns else 0
        avg_touches_total = df['Touches_Touches'].mean()
        possession_control_index = ((avg_touches_att + avg_carries_third) / avg_touches_total) if avg_touches_total > 0 else 0
        
        # Índice de transición
        avg_prgp = df['PrgP'].mean()
        avg_prgc = df['Carries_PrgC'].mean()
        avg_poss_raw = df['Poss'].mean()
        transition_index = ((avg_prgp + avg_prgc) / avg_poss_raw) if avg_poss_raw > 0 else 0
        
        # ✅ RETORNAR TODAS LAS MÉTRICAS (23 valores)
        return (
            avg_ck,
            var_ck,  # 0 - ✅ CAMBIADO: varianza en vez de promedio
            avg_xg,  # 1
            avg_sca,  # 2
            avg_cross,  # 3
            avg_poss,  # 4
            avg_att_3rd,  # 5
            avg_gf,  # 6
            avg_ga,  # 7
            sh_accuracy,  # 8
            xg_shot,  # 9
            attacking_presence,  # 10
            possession_shot,  # 11
            progressive_pass_ratio,  # 12
            final_third_involvement,  # 13
            assist_sca,  # 14
            creative_efficiency,  # 15
            high_press_intensity,  # 16
            interception_tackle,  # 17
            clearance_ratio,  # 18
            progressive_carry_ratio,  # 19
            carry_pass_balance,  # 20
            offensive_index,  # 21
            transition_index  # 22
        )
    
    # ===========================
    # PROMEDIOS DE LIGA (is_team=False)
    # ===========================
    
    avg_cross = df['Performance_Crs'].mean()
    avg_att_3rd = df['Touches_Att 3rd'].mean()
    avg_sca = df['SCA Types_SCA'].mean()
    avg_xg = df['Expected_xG'].mean()
    
    # ✅ CAMBIO: VARIANZA EN VEZ DE PROMEDIO DE CK
    var_ck = df['Pass Types_CK'].var() if len(df) > 1 else 0
    avg_ck = df['Pass Types_CK'].mean()
    
    avg_gf = df['GF'].mean()
    avg_ga = df['GA'].mean()
    
    # ✅ AGREGAR MÉTRICAS BÁSICAS PARA NORMALIZACIÓN
    avg_sh = df['Standard_Sh'].mean() if 'Standard_Sh' in df.columns else 0
    
    return (
        
        var_ck,  # 0 - ✅ CAMBIADO
        avg_xg,  # 1
        avg_sca,  # 2
        avg_cross,  # 3
        avg_att_3rd,  # 4
        avg_gf,  # 5
        avg_ga,  # 6
        avg_sh,  # 7 - NUEVO
        avg_ck
    )

# ===========================
# PROCESAMIENTO DE DATOS CON PPP Y NUEVAS MÉTRICAS
# ===========================

lst_years = ["1819", "1920", "2021", "2122", "2223", "2324", "2425", "2526"]
lst_data = []
y = []

USE_ONE_HOT_ENCODING = True

print("\n🔄 PROCESANDO DATOS CON PPP Y MÉTRICAS AVANZADAS...")
print("=" * 80)

for i in lst_1:
    
    if i[2] < 5:
        continue
    
    local = i[0]
    away = i[1]
    round_num = i[2]
    season = i[3]
    date = i[4]
    league_code = i[5]
    
    dic_df = {}
    
    # Promedios de liga
    lst_avg = get_average(
        df_database[
            (df_database['season'] == season) & 
            (df_database['round'] < round_num) &
            (df_database['league'] == league_code)
        ],
        is_team=False
    )
    
    # ✅ FUNCIÓN MEJORADA: Maneja métricas originales y avanzadas
    def create_line(df, is_form=True, is_team=False, use_advanced=True):
        """
        Args:
            df: DataFrame con datos del equipo
            is_form: Si True, toma solo últimos 8 partidos
            is_team: Si True, normaliza contra promedios de liga
            use_advanced: Si True, incluye métricas avanzadas (23 valores)
                         Si False, solo métricas originales (8 valores)
        """
        if is_form:
            df = df[-6:]
        
        if use_advanced:
            # Retorna 23 valores (todas las métricas)
            return get_average(df, is_team, lst_avg)
        else:
            # Retorna solo 8 valores originales
            result = get_average(df, is_team, lst_avg)
            return result[:9]  # Primeros 8 valores
    
    # Extraer DataFrames
    (team1_home, team1_away, team1_opp_home, team1_opp_away,
     team2_home, team2_away, team2_opp_home, team2_opp_away) = get_dataframes(
        df_database, season, round_num, local, away, league=league_code
    )
    
    # Corners reales
    ck = get_ck(df_database, season, round_num, local, away, league=league_code)
    y.append(ck)
    
    # Head to Head
    index = lst_years.index(season)
    result = lst_years[:index+1]
    team1_h2h, team2_h2h = get_head_2_head(
        df_database, local, away, seasons=result, league=league_code
    )
    
    # ✅ PPP
    local_ppp = get_team_ppp(df_database, local, season, round_num, league=league_code)
    away_ppp = get_team_ppp(df_database, away, season, round_num, league=league_code)
    ppp_diff = local_ppp - away_ppp
    
    dic_df['ppp_local'] = (local_ppp,)
    dic_df['ppp_away'] = (away_ppp,)
    dic_df['ppp_difference'] = (ppp_diff,)
    
    # ✅ FEATURES CON MÉTRICAS AVANZADAS (23 valores cada una)
    dic_df['lst_team1_home_form'] = create_line(team1_home, True, True, use_advanced=True)
    dic_df['lst_team1_home_general'] = create_line(team1_home, False, True, use_advanced=True)
    dic_df['lst_team1_away_form'] = create_line(team1_away, True, True, use_advanced=True)
    dic_df['lst_team1_away_general'] = create_line(team1_away, False, True, use_advanced=True)
    
    dic_df['lst_team2_home_form'] = create_line(team2_home, True, True, use_advanced=True)
    dic_df['lst_team2_home_general'] = create_line(team2_home, False, True, use_advanced=True)
    dic_df['lst_team2_away_form'] = create_line(team2_away, True, True, use_advanced=True)
    dic_df['lst_team2_away_general'] = create_line(team2_away, False, True, use_advanced=True)
    
    dic_df['lst_team1_h2h'] = create_line(team1_h2h, False, True, use_advanced=True)
    dic_df['lst_team2_h2h'] = create_line(team2_h2h, False, True, use_advanced=True)
    
    # ✅ FEATURES CON MÉTRICAS ORIGINALES (8 valores) - SOLO PARA OPONENTES
    dic_df['lst_team1_opp_away'] = create_line(team1_opp_away, False, True, use_advanced=False)
    dic_df['lst_team2_opp_home'] = create_line(team2_opp_home, False, True, use_advanced=False)
    
    # One-Hot Encoding
    if USE_ONE_HOT_ENCODING:
        league_dummies = {
            'league_ESP': 1 if league_code == 'ESP' else 0,
            'league_GER': 1 if league_code == 'GER' else 0,
            'league_FRA': 1 if league_code == 'FRA' else 0,
            'league_ITA': 1 if league_code == 'ITA' else 0,
            'league_NED': 1 if league_code == 'NED' else 0
        }
        
        for key, value in league_dummies.items():
            dic_df[key] = (value,)
    
    # ✅ CONSTRUIR VECTOR DE FEATURES CON NOMBRES DESCRIPTIVOS
    lst_base_advanced = [
        "avg_ck","var_ck",  # ✅ CAMBIADO
        "xg", "sca", "cross", "poss", "att_3rd", "gf", "ga",
        "sh_accuracy", "xg_shot", "attacking_presence", "possession_shot",
        "progressive_pass_ratio", "final_third_involvement", "assist_sca", "creative_efficiency",
        "high_press_intensity", "interception_tackle", "clearance_ratio",
        "progressive_carry_ratio", "carry_pass_balance", "offensive_index", "transition_index"
    ]
    
    lst_base_original = [
        "var_ck","xg", "sca", "cross", "poss", "att_3rd", "gf", "ga","avg_ck"
    ]
    
    lst_features_values = []
    lst_features_names = []
    
    for key in dic_df:
        lst_features_values.extend(list(dic_df[key]))
        
        # Casos especiales
        if key in ['ppp_local', 'ppp_away', 'ppp_difference']:
            lst_features_names.append(key)
        elif key.startswith('league_'):
            lst_features_names.append(key)
        elif key in ['lst_team1_opp_away', 'lst_team2_opp_home']:
            # ✅ Métricas ORIGINALES (8 valores)
            lst_features_names.extend([f"{key}_{col}" for col in lst_base_original])
        else:
            # ✅ Métricas AVANZADAS (23 valores)
            lst_features_names.extend([f"{key}_{col}" for col in lst_base_advanced])
    
    lst_data.append(lst_features_values)

# ===========================
# CREAR DATAFRAME FINAL
# ===========================

df_data = pd.DataFrame(data=lst_data, columns=lst_features_names)

print(f"\n✅ PROCESAMIENTO COMPLETADO:")
print(f"   Shape inicial: {df_data.shape}")
print(f"   Total partidos: {len(df_data)}")
print(f"   Features totales: {df_data.shape[1]}")

# ===========================
# LIMPIEZA DE DATOS NULOS
# ===========================

print(f"\n🧹 LIMPIANDO DATOS NULOS...")

import numpy as np
nulos_antes_X = df_data.isnull().sum().sum()
nulos_antes_y = np.isnan(y).sum() if isinstance(y, np.ndarray) else sum(pd.isna(y))

print(f"   Nulos en X (antes): {nulos_antes_X}")
print(f"   Nulos en Y (antes): {nulos_antes_y}")

y_array = np.array(y).flatten()

mask_valid_X = ~df_data.isnull().any(axis=1)
mask_valid_y = ~np.isnan(y_array)
mask_combined = mask_valid_X & mask_valid_y

df_data = df_data[mask_combined].reset_index(drop=True)
y_array = y_array[mask_combined]

print(f"\n✅ LIMPIEZA COMPLETADA:")
print(f"   Nulos en X (después): {df_data.isnull().sum().sum()}")
print(f"   Nulos en Y (después): {np.isnan(y_array).sum()}")
print(f"   Filas eliminadas: {len(mask_combined) - mask_combined.sum()}")
print(f"   Shape final: {df_data.shape}")

# ===========================
# VERIFICACIÓN FINAL
# ===========================

print(f"\n🔍 VERIFICACIÓN DE NUEVAS FEATURES:")
print(f"   ✅ Features con 'var_ck': {len([c for c in df_data.columns if 'var_ck' in c])}")
print(f"   ✅ Features con métricas avanzadas: {len([c for c in df_data.columns if any(m in c for m in ['sh_accuracy', 'offensive_index'])])}")
print(f"   ✅ Features de oponentes (8 valores): {len([c for c in df_data.columns if 'opp' in c])}")

y = y_array.tolist()

print("\n" + "=" * 80)
print("✅ PROCESO COMPLETADO - DATOS LISTOS PARA ENTRENAMIENTO")
print("=" * 80)


🔄 PROCESANDO DATOS CON PPP Y MÉTRICAS AVANZADAS...

✅ PROCESAMIENTO COMPLETADO:
   Shape inicial: (11419, 266)
   Total partidos: 11419
   Features totales: 266

🧹 LIMPIANDO DATOS NULOS...
   Nulos en X (antes): 3133
   Nulos en Y (antes): 0

✅ LIMPIEZA COMPLETADA:
   Nulos en X (después): 0
   Nulos en Y (después): 0
   Filas eliminadas: 390
   Shape final: (11029, 266)

🔍 VERIFICACIÓN DE NUEVAS FEATURES:
   ✅ Features con 'var_ck': 12
   ✅ Features con métricas avanzadas: 20
   ✅ Features de oponentes (8 valores): 18

✅ PROCESO COMPLETADO - DATOS LISTOS PARA ENTRENAMIENTO


In [21]:
df_database = pd.read_csv("data/database_4_ligas.csv")
df_database["season"] = df_database["season"].astype(str)
df_database_export = df_database.copy()
df_database_export = df_database_export.drop_duplicates(subset=["game", "league"])
df_database_export = df_database_export[["local", "away", "round", "season", "date", "league"]]

lst_1 = df_database_export.values.tolist()

# Excluir temporada 1718 si es necesario
lst_1 = [row for row in lst_1 if row[3] != "1718"]

In [22]:
lst_1[0][3]

'1819'

In [27]:
partidos = [
    ("Elche", "Real Sociedad"),
    ("Girona", "Alavés"),
    ("Sevilla", "Osasuna"),
    ("Atlético Madrid", "Levante"),
    ("Espanyol", "Villarreal"),
    ("Athletic Club", "Oviedo"),
    ("Rayo Vallecano", "Real Madrid"),
    ("Mallorca", "Getafe"),
    ("Valencia", "Betis"),
    ("Barcelona", "Elche"),
    ("Betis", "Mallorca"),
    ("Barcelona", "Celta Vigo")
]

for i in partidos:
    resultado = predecir_corners(
    local=i[0],
    visitante=i[1], 
    jornada=12,
    temporada="2526",
    league_code="ESP"  # ✅ NUEVO PARÁMETRO
)
    



🏟️  Elche vs Real Sociedad
📅 Temporada 2526 | Jornada 12 | Liga: ESP

🎲 PREDICCIÓN MODELO: 7.96 corners totales
   PPP: Elche (1.40) vs Real Sociedad (1.09) | Diff: +0.31

📊 ESTADÍSTICAS HISTÓRICAS:
   Elche (Casa): 4.2 CK/partido | xG: 1.54 | Poss: 64.8%
   Real Sociedad (Fuera): 6.6 CK/partido | xG: 1.02 | Poss: 47.4%
   Corners recibidos: Elche (6.4) | Real Sociedad (4.0)
   Total esperado (suma): 10.8 corners

🔄 HEAD TO HEAD (últimos 3 partidos):
   Elche: 2.3 CK/partido
   Real Sociedad: 6.0 CK/partido
   Promedio total: 8.3 corners

📈 PROBABILIDADES EXACTAS (Poisson):
    3 corners:  2.93% █
    4 corners:  5.83% ██
    5 corners:  9.28% ████
    6 corners: 12.32% ██████
    7 corners: 14.02% ███████ ⭐
    8 corners: 13.96% ██████
    9 corners: 12.35% ██████
   10 corners:  9.84% ████
   11 corners:  7.12% ███
   12 corners:  4.73% ██
   13 corners:  2.90% █

✅ Valor más probable: 7 corners (14.02%)
📊 Rango 80% confianza: 4-11 corners

🎯 ANÁLISIS OVER/UNDER:
Línea      Prob Ove

Traceback (most recent call last):
  File "c:\Users\Danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\internals\construction.py", line 939, in _finalize_columns_and_data
    columns = _validate_or_indexify_columns(contents, columns)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\internals\construction.py", line 986, in _validate_or_indexify_columns
    raise AssertionError(
AssertionError: 266 columns passed, passed data had 264 columns

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Danie\AppData\Local\Temp\ipykernel_20264\2825540969.py", line 382, in predecir_corners
    df_input = pd.DataFrame([lst_features_values], columns=lst_features_names)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Danie\AppData\Local\Programs\Python\Pyt

           Equipo  Partidos  Media_CK  Mediana_CK  Std_CK  CV_%  \
0         Arsenal        10      7.30         8.0    3.10  42.5   
1     Aston Villa        10      5.30         5.0    2.45  46.3   
2     Bournemouth         9      5.56         5.0    1.57  28.3   
3       Brentford        10      5.30         5.0    2.53  47.8   
4        Brighton        10      4.80         4.0    2.18  45.5   
5         Burnley        10      2.60         2.5    1.50  57.6   
6         Chelsea        10      6.10         5.5    2.34  38.4   
7  Crystal Palace        10      3.80         3.5    2.27  59.8   
8         Everton         9      4.11         3.0    2.96  72.0   
9          Fulham        10      5.10         3.5    2.77  54.4   

   Pct_Cerca_Media  Cambios_Bruscos_%   IQR  Rango  ...  Min  Max  Outliers  \
0             40.0               22.2  5.00     10  ...    2   12         0   
1             50.0               22.2  2.75      8  ...    2   10         0   
2             77.8       

In [42]:
def kelly_fraction(p, odds, fraction=0.2):
    b = odds - 1
    q = 1 - p
    f_star = (b * p - q) / b
    f_star = max(f_star, 0)  # evita negativos
    return f_star * fraction  # usa 0.1 para Kelly 10%

kelly_fraction(.60, 1.83, fraction=0.2)


0.023614457831325295

In [44]:
partidos = [
    ("Werder Bremen", "Wolfsburg"),
    ("Hoffenheim", "RB Leipzig"),
    ("Leverkusen", "Heidenheim"),
    ("Hamburger SV", "Dortmund"),
    ("Union Berlin", "Bayern"),
    ("Gladbach", "Köln"),
    ("Freiburg", "St. Pauli"),
    ("Stuttgart", "Augsburg"),
    ("Eint Frankfurt", "Mainz 05")

]

for i in partidos:
    resultado = predecir_corners(
    local=i[0],
    visitante=i[1], 
    jornada=10,
    temporada="2526",
    league_code="GER"  # ✅ NUEVO PARÁMETRO
)


🏟️  Werder Bremen vs Wolfsburg
📅 Temporada 2526 | Jornada 10 | Liga: GER

🎲 PREDICCIÓN MODELO: 9.22 corners totales
   PPP: Werder Bremen (1.33) vs Wolfsburg (1.00) | Diff: +0.33

📊 ESTADÍSTICAS HISTÓRICAS:
   Werder Bremen (Casa): 3.5 CK/partido | xG: 1.65 | Poss: 51.0%
   Wolfsburg (Fuera): 3.5 CK/partido | xG: 1.20 | Poss: 46.8%
   Corners recibidos: Werder Bremen (7.4) | Wolfsburg (8.2)
   Total esperado (suma): 7.0 corners

🔄 HEAD TO HEAD (últimos 3 partidos):
   Werder Bremen: 5.0 CK/partido
   Wolfsburg: 4.0 CK/partido
   Promedio total: 9.0 corners

📈 PROBABILIDADES EXACTAS (Poisson):
    4 corners:  2.98% █
    5 corners:  5.49% ██
    6 corners:  8.44% ████
    7 corners: 11.12% █████
    8 corners: 12.82% ██████
    9 corners: 13.14% ██████ ⭐
   10 corners: 12.12% ██████
   11 corners: 10.16% █████
   12 corners:  7.81% ███
   13 corners:  5.54% ██
   14 corners:  3.65% █

✅ Valor más probable: 9 corners (13.14%)
📊 Rango 80% confianza: 6-13 corners

🎯 ANÁLISIS OVER/UNDER:
L

Traceback (most recent call last):
  File "c:\Users\Danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\internals\construction.py", line 939, in _finalize_columns_and_data
    columns = _validate_or_indexify_columns(contents, columns)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Danie\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\internals\construction.py", line 986, in _validate_or_indexify_columns
    raise AssertionError(
AssertionError: 266 columns passed, passed data had 264 columns

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\Danie\AppData\Local\Temp\ipykernel_20264\2825540969.py", line 382, in predecir_corners
    df_input = pd.DataFrame([lst_features_values], columns=lst_features_names)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Danie\AppData\Local\Programs\Python\Pyt


❌ ERROR: 266 columns passed, passed data had 260 columns

🏟️  Gladbach vs Köln
📅 Temporada 2526 | Jornada 10 | Liga: GER

🎲 PREDICCIÓN MODELO: 11.19 corners totales
   PPP: Gladbach (0.67) vs Köln (1.38) | Diff: -0.71

📊 ESTADÍSTICAS HISTÓRICAS:
   Gladbach (Casa): 5.0 CK/partido | xG: 1.12 | Poss: 45.0%
   Köln (Fuera): 4.0 CK/partido | xG: 1.28 | Poss: 43.4%
   Corners recibidos: Gladbach (5.8) | Köln (5.0)
   Total esperado (suma): 9.0 corners

🔄 HEAD TO HEAD (últimos 3 partidos):
   Gladbach: 4.0 CK/partido
   Köln: 7.3 CK/partido
   Promedio total: 11.3 corners

📈 PROBABILIDADES EXACTAS (Poisson):
    6 corners:  3.76% █
    7 corners:  6.01% ███
    8 corners:  8.41% ████
    9 corners: 10.46% █████
   10 corners: 11.71% █████
   11 corners: 11.92% █████ ⭐
   12 corners: 11.12% █████
   13 corners:  9.57% ████
   14 corners:  7.65% ███
   15 corners:  5.71% ██
   16 corners:  3.99% █

✅ Valor más probable: 11 corners (11.92%)
📊 Rango 80% confianza: 7-15 corners

🎯 ANÁLISIS OVER/

In [28]:
partidos = [
    ("Como", "Cagliari"),
    ("Lecce", "Hellas Verona"),
    ("Juventus", "Torino"),
    ("Parma", "Milan")
    
]

for i in partidos:
    resultado = predecir_corners(
    local=i[0],
    visitante=i[1], 
    jornada=11,
    temporada="2526",
    league_code="ITA"  # ✅ NUEVO PARÁMETRO
)
    
    '''
    ("Fiorentina", "Lecce"),
    ("Torino", "Pisa"),
    ("Parma", "Bologna"),
    ("Milan", "Roma"),
    '''


🏟️  Como vs Cagliari
📅 Temporada 2526 | Jornada 11 | Liga: ITA

🎲 PREDICCIÓN MODELO: 9.33 corners totales
   PPP: Como (1.70) vs Cagliari (1.00) | Diff: +0.70

📊 ESTADÍSTICAS HISTÓRICAS:
   Como (Casa): 3.0 CK/partido | xG: 1.48 | Poss: 57.6%
   Cagliari (Fuera): 2.8 CK/partido | xG: 1.00 | Poss: 45.5%
   Corners recibidos: Como (4.8) | Cagliari (5.0)
   Total esperado (suma): 5.8 corners

🔄 HEAD TO HEAD (últimos 2 partidos):
   Como: 5.5 CK/partido
   Cagliari: 3.0 CK/partido
   Promedio total: 8.5 corners

📈 PROBABILIDADES EXACTAS (Poisson):
    4 corners:  2.80% █
    5 corners:  5.22% ██
    6 corners:  8.12% ████
    7 corners: 10.82% █████
    8 corners: 12.63% ██████
    9 corners: 13.10% ██████ ⭐
   10 corners: 12.22% ██████
   11 corners: 10.37% █████
   12 corners:  8.07% ████
   13 corners:  5.79% ██
   14 corners:  3.86% █

✅ Valor más probable: 9 corners (13.10%)
📊 Rango 80% confianza: 6-13 corners

🎯 ANÁLISIS OVER/UNDER:
Línea      Prob Over    Cuota Impl   Confianza    

In [ ]:
partidos = [
    ("Köln", "Hamburger"),
    ("Wolfsburg", "Hoffenheim"),
]

for i in partidos:
    resultado = predecir_corners(
    local=i[0],
    visitante=i[1], 
    jornada=9,
    temporada="2526",
    league_code="GER"  # ✅ NUEVO PARÁMETRO
)
    

partidos = [
    ("Hellas Verona", "Inter"),
    ("Fiorentina", "Lecce"),
    ("Torino", "Pisa"),
    ("Parma", "Bologna"),
    ("Milan", "Roma"),
]

for i in partidos:
    resultado = predecir_corners(
    local=i[0],
    visitante=i[1], 
    jornada=10,
    temporada="2526",
    league_code="ITA"  # ✅ NUEVO PARÁMETRO
)
    
partidos = [
    ("Heracles Almelo", "Zwolle"),
    ("Groningen", "Twente"),
    ("Sparta R'dam", "AZ Alkmaar"),
    ("Utrecht", "NEC Nijmegen")
]

for i in partidos:
    resultado = predecir_corners(
    local=i[0],
    visitante=i[1], 
    jornada=11,
    temporada="2526",
    league_code="NED"  # ✅ NUEVO PARÁMETRO
)
    
partidos = [
    ("Toulouse", "Le Havre"),
    ("Lens", "Lorient"),
    ("Rennes", "Strasbourg"),
    ("Lille", "Angers"),
    ("Nantes", "Metz"),
    ("Lyon", "Brest")
]
    
for i in partidos:
    resultado = predecir_corners(
    local=i[0],
    visitante=i[1], 
    jornada=11,
    temporada="2526",
    league_code="NED")